## IMPORTS

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
#from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

import pandas as pd

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
books = pd.read_csv("books_cleaned.csv")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


### Loading vector search

In [4]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: str

In [5]:
# Convert dataset as txt file as langchain textloader does not work with pandas df
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False)

In [14]:
# load data in textloader
raw_documents = TextLoader("tagged_description.txt").load()

# instantiate text splitter
text_splitter = CharacterTextSplitter(chunk_size=0.1, chunk_overlap=0, separator="\n")

# apply it to the documents loaded
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [7]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

### Create word embeddings and insert it to vector database

In [8]:
db_books = Chroma.from_documents(
    documents,
    embedding=HuggingFaceEmbeddings()
)

/home/itzjmbruhhh/Projects/book-recommender/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Testing our vector search

In [9]:
# Query to specify which books to find
query = "A book to teach children about nature"

# similarity search using the query, that shows 10 results
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='bdcf393b-7014-44b6-947a-9789ea97ba9e', metadata={'source': 'tagged_description.txt'}, page_content='"9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson v

### Since we want to get titles instead of actual descriptions
### We will use the isbn13 identifiers that we put in front of our descriptions

In [24]:
# Pandas code to strip tagged_description to get the isbn13 in front, so that it only shows that specific book entry
books[books["isbn13"] == int(docs[0].page_content.strip('"').split()[0])]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...


In [26]:
# Function to return all recommendation from a query
def retrieve_semantic_recommendations(
        query : str, # Query
        top_k : int = 10, # Number of results
) -> pd.DataFrame: # Return pandas dataframe
    recs = db_books.similarity_search(query, k=50) # The code to give us recommendations

    books_list = []

    for i in range(0, len(recs)): # Loop over all recommendations returned
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [28]:
# Testing our query
retrieve_semantic_recommendations("A book about world war 2")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
66,9780007162994,0007162995,If I Die in a Combat Zone,Tim O'Brien,"Vietnam War, 1961-1975",http://books.google.com/books/content?id=0qUtS...,Perhaps the best book to emerge from the Vietn...,2003.0,3.95,208.0,11.0,If I Die in a Combat Zone,9780007162994 Perhaps the best book to emerge ...
121,9780060512804,0060512806,Cryptonomicon,Neal Stephenson,Fiction,http://books.google.com/books/content?id=qYAmf...,With this extraordinary first volume in what p...,2002.0,4.25,1139.0,80225.0,Cryptonomicon,9780060512804 With this extraordinary first vo...
234,9780060792176,0060792175,Everything Is Illuminated tie-in,Jonathan Safran Foer,Fiction,http://books.google.com/books/content?id=tsO2v...,"With only a yellowing photograph in hand, a yo...",2005.0,3.90,276.0,1754.0,Everything Is Illuminated tie-in: A Novel,9780060792176 With only a yellowing photograph...
330,9780060975098,0060975091,Three Farmers on Their Way to a Dance,Richard Powers,Fiction,http://books.google.com/books/content?id=U6zjw...,"In the spring of 1914, renowned photographer A...",1992.0,3.84,352.0,709.0,Three Farmers on Their Way to a Dance,"9780060975098 In the spring of 1914, renowned ..."
524,9780099483472,0099483475,All Quiet on the Western Front,Erich Maria Remarque,"World War, 1914-1918",NaN,All Quiet on the Western Front is probably the...,2005.0,3.95,216.0,1018.0,All Quiet on the Western Front,9780099483472 All Quiet on the Western Front i...
541,9780099910107,0099910101,A Farewell to Arms,Ernest Hemingway,War,http://books.google.com/books/content?id=m68Lh...,"In 1918 Ernest Hemingway went to war, to the '...",1994.0,3.80,293.0,210197.0,A Farewell to Arms,9780099910107 In 1918 Ernest Hemingway went to...
560,9780140110876,0140110879,You Bright and Risen Angels,William T. Vollmann,Fiction,http://books.google.com/books/content?id=d0buA...,This comic and surreal novel about the beastli...,1988.0,4.08,635.0,747.0,You Bright and Risen Angels: A Cartoon,9780140110876 This comic and surreal novel abo...
572,9780140149241,0140149244,We Were the Rats,Lawson Glassop,"Tobruk, Battles of, 1941-1942",NaN,Reissue of the famous novel based on the autho...,1991.0,3.23,275.0,13.0,We Were the Rats,9780140149241 Reissue of the famous novel base...
580,9780140158762,0140158766,Chickenhawk - Back in the World,Robert Mason,Biography & Autobiography,http://books.google.com/books/content?id=R7cGA...,From the author of the bestselling autobiograp...,1994.0,4.05,400.0,166.0,Chickenhawk - Back in the World: Life After Vi...,9780140158762 From the author of the bestselli...
795,9780142002803,0142002801,"The Fall of Berlin, 1945",Antony Beevor,History,http://books.google.com/books/content?id=u6avD...,Chronicles the horror of Berlin's fall to the ...,2003.0,4.28,490.0,9635.0,"The Fall of Berlin, 1945",9780142002803 Chronicles the horror of Berlin'...
